# Verificaremos si el modelo que estoy usando es el correcto

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
import cv2
import numpy as np
import glob
import os
from random import shuffle

In [2]:
X_train=[]
X_test=[]
Y_test=[]
Y_train=[]
dataTr=[]
for filename in glob.glob(os.path.join('data/train/malignant','*.jpg')):
    dataTr.append([1,cv2.imread(filename)])
for filename in glob.glob(os.path.join('data/train/benign','*.jpg')):
    dataTr.append([0,cv2.imread(filename)])

In [3]:
shuffle(dataTr)
print("en total tenemos: "+str(len(dataTr))+ " imagenes dentro de la carpeta train")

en total tenemos: 2637 imagenes dentro de la carpeta train


In [4]:
def creacion_modelo():
    modelo=Sequential()
    modelo.add(Convolution2D(32,(3,3),input_shape=(224,224,3),activation='relu'))
    modelo.add(MaxPooling2D(pool_size=((2,2))))
    modelo.add(Flatten())
    modelo.add(Dense(128,activation='relu'))
    modelo.add(Dense(50,activation='relu'))
    modelo.add(Dense(1,activation='sigmoid'))
    return modelo

## metodos que ayudaran en el proceso de entrenamiento

In [5]:
def evaluacion(modelo,porcentaje,nombre,x_e,y_e,x_p,y_p):
    prediccion=0
    epocas=1
    while(True):
        if prediccion < porcentaje:
            prediccion=(validacion(x_e,y_e,x_p,y_p,modelo,1))*100
            epocas +=1
        else:
            print("==> Para el metodo "+nombre+" se utilizo: "+str(epocas-1)+" epocas para llegar a mas del "+str(porcentaje)+"% de acertividad")
            break
    return (epocas, prediccion)

In [6]:
def validacion(x_e,y_e,x_p,y_p,model,epocas):
    entre=model.fit(x_e,y_e,batch_size=32,epochs=epocas,validation_data=(x_p,y_p))
    a=model.evaluate(x_p,y_p)
    return a[1]

## creacion de modelos para cada prueba

In [7]:
from tensorflow import keras

In [8]:
opt = keras.optimizers.Adam(learning_rate=0.0009)
modelo1=creacion_modelo()
modelo1.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
modelo2=creacion_modelo()
modelo2.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
modelo3=creacion_modelo()
modelo3.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
modelo4=creacion_modelo()
modelo4.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
modelo5=creacion_modelo()
modelo5.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
def particion(datos):
    imagenes=[]
    etiquetas=[]
    for i,j in datos:
        imagenes.append(j)
        etiquetas.append(i)
    imagenes=np.array(imagenes)
    etiquetas=np.array(etiquetas)
    return (imagenes,etiquetas)

## validacion Cruzada

In [10]:
n=int(len(dataTr)/5)
output=[dataTr[i:i + n] for i in range(0, len(dataTr), n)]
porcion1=output[0]
porcion2=output[1]
porcion3=output[2]
porcion4=output[3]
porcion5=output[4]
porcion5.extend(output[5])
print(len(porcion1),len(porcion2),len(porcion3),len(porcion4),len(porcion5))

527 527 527 527 529


In [11]:
x_p1,y_p1=particion(porcion1)
x_p2,y_p2=particion(porcion2)
x_p3,y_p3=particion(porcion3)
x_p4,y_p4=particion(porcion4)
x_p5,y_p5=particion(porcion5)

In [12]:
x_entre=[]
y_entre=[]
entre1=porcion4+porcion3+porcion2+porcion1
entre2=porcion5+porcion3+porcion2+porcion1
entre3=porcion5+porcion4+porcion2+porcion1
entre4=porcion5+porcion4+porcion3+porcion1
entre5=porcion5+porcion4+porcion3+porcion2

In [ ]:
vector_epocas=[]
vector_aserti=[]
x_entre,y_entre=particion(entre1)
val_epocas, val_asertividad=evaluacion(modelo1,80,"ADAM",x_entre,y_entre,x_p5,y_p5)
vector_epocas.append(val_epocas)
vector_aserti.append(val_asertividad)
x_entre=[]
y_entre=[]

Train on 2108 samples, validate on 529 samples


In [ ]:
x_entre,y_entre=particion(entre2)
val_epocas, val_asertividad=evaluacion(modelo2,80,"ADAM",x_entre,y_entre,x_p4,y_p4)
vector_epocas.append(val_epocas)
vector_aserti.append(val_asertividad)
x_entre=[]
y_entre=[]

In [ ]:
x_entre,y_entre=particion(entre3)
val_epocas, val_asertividad=evaluacion(modelo3,80,"ADAM",x_entre,y_entre,x_p3,y_p3)
vector_epocas.append(val_epocas)
vector_aserti.append(val_asertividad)
x_entre=[]
y_entre=[]

In [ ]:
x_entre,y_entre=particion(entre4)
val_epocas, val_asertividad=evaluacion(modelo4,80,"ADAM",x_entre,y_entre,x_p2,y_p2)
vector_epocas.append(val_epocas)
vector_aserti.append(val_asertividad)
x_entre=[]
y_entre=[]

In [ ]:
x_entre,y_entre=particion(entre5)
val_epocas, val_asertividad=evaluacion(modelo5,80,"ADAM",x_entre,y_entre,x_p1,y_p1)
vector_epocas.append(val_epocas)
vector_aserti.append(val_asertividad)

In [ ]:
media = np.mean(vector_aserti)
desviacion_estandart = np.std(vector_aserti)
print(vector_aserti)
print(vector_epocas)
print("se tiene que como media el valor: "+str(media))
print("se tiene que como desviacion estandart: "+str(desviacion_estandart))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as mp

In [ ]:
mp.plot([0.72778827, 0.84440225, 0.91081595, 0.9487666, 0.9411765],[1,2,3,4,5],"gs")